In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
import pandas as pd

In [2]:
MODEL_PATH = Path.cwd()/'models'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class CustomDataset(Dataset):
    def __init__(self, df, transform=None, target_transform=None):
        self.df = df
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.df)


    def __getitem__(self, idx):
        x = self.df.drop(labels='AGE', axis=1).iloc[idx]
        x = torch.from_numpy(x.values)
        y = self.df.iloc[idx].AGE
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
        return x, y

In [4]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        
        self.arch = nn.Sequential(
            nn.Linear(20, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 1)
        )
    def forward(self, x):
        return self.arch(x)

In [5]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, in_5_range = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device).float().unsqueeze(dim=1)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            in_5_range += (abs(pred - y) < 5).sum().item()
    test_loss /= num_batches
    in_5_range /= size
    print(f"Test Error: \n Predictions in 5 range: {(100*in_5_range):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device).float(), y.to(device).float().unsqueeze(dim=1)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
train_df = pd.read_csv('Data/train_data.csv')
test_df = pd.read_csv('Data/test_data.csv')

In [8]:
train_set = CustomDataset(train_df)
test_set = CustomDataset(test_df)

In [9]:
train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)

In [10]:
model = NN().to(device)

In [11]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 45.353626  [    0/20081]
loss: 55.969151  [ 3200/20081]
loss: 49.031498  [ 6400/20081]
loss: 46.142799  [ 9600/20081]
loss: 45.317604  [12800/20081]
loss: 44.647869  [16000/20081]
loss: 30.156399  [19200/20081]
Test Error: 
 Predictions in 5 range: 13.3%, Avg loss: 25.065018 

Epoch 2
-------------------------------
loss: 24.717577  [    0/20081]
loss: 15.154387  [ 3200/20081]
loss: 14.975642  [ 6400/20081]
loss: 13.357244  [ 9600/20081]
loss: 15.380359  [12800/20081]
loss: 18.183578  [16000/20081]
loss: 16.570034  [19200/20081]
Test Error: 
 Predictions in 5 range: 22.0%, Avg loss: 14.016928 

Epoch 3
-------------------------------
loss: 12.867839  [    0/20081]
loss: 15.354355  [ 3200/20081]
loss: 14.309418  [ 6400/20081]
loss: 21.544062  [ 9600/20081]
loss: 16.246109  [12800/20081]
loss: 13.728454  [16000/20081]
loss: 11.293884  [19200/20081]
Test Error: 
 Predictions in 5 range: 22.8%, Avg loss: 13.294369 

Epoch 4
--------------------

In [20]:
torch.save(model, MODEL_PATH/'pytorch200_100.pth')